In [ ]:
from pathlib import Path

import pandas as pd

from analytics.app.data.load import list_pipelines, load_pipeline_logs
from analytics.app.data.transform import (
    df_aggregate_eval_metric,
    dfs_models_and_evals,
    patch_yearbook_time,
    pipeline_leaf_times_df,
)
from analytics.plotting.common.save import save_plot
from analytics.plotting.common.tradeoff_scatterplot import plot_tradeoff_scatter
from modyn.supervisor.internal.grpc.enums import PipelineStage
from modyn.supervisor.internal.pipeline_executor.models import PipelineLogs

%load_ext autoreload
%autoreload 2

In [ ]:
pipelines_dirs = [
    Path("/Users/robinholzinger/robin/dev/eth/modyn-robinholzi-data/data/triggering/yearbook/10_baselines_time"),
    Path("/Users/robinholzinger/robin/dev/eth/modyn-robinholzi-data/data/triggering/yearbook/11_baselines_amount"),
    Path("/Users/robinholzinger/robin/dev/eth/modyn-robinholzi-data/data/triggering/yearbook/20_datadrift_static"),
    Path("/Users/robinholzinger/robin/dev/eth/modyn-robinholzi-data/data/triggering/yearbook/21_datadrift_dynamic"),
    Path(
        "/Users/robinholzinger/robin/dev/eth/modyn-robinholzi-data/data/triggering/yearbook/30_performance/num_misclass"
    ),
    Path(
        "/Users/robinholzinger/robin/dev/eth/modyn-robinholzi-data/data/triggering/yearbook/30_performance/static_dyn"
    ),
    Path(
        "/Users/robinholzinger/robin/dev/eth/modyn-robinholzi-data/data/triggering/yearbook/40_cost_dataincorporation_latency"
    ),
    Path(
        "/Users/robinholzinger/robin/dev/eth/modyn-robinholzi-data/data/triggering/yearbook/41_avoidable_miclass_cost"
    ),
]

pipeline_logs: dict[int, PipelineLogs] = {}
pipelines: dict[int, tuple[str, Path]] = {}

for dir in pipelines_dirs:
    print("Reading", dir)
    dir_pipelines = list_pipelines(dir)
    print(dir_pipelines)
    pipelines.update(dir_pipelines)

    max_pipeline_id = max(dir_pipelines.keys())
    print(pipelines)
    pipeline_logs.update({p_id: load_pipeline_logs(p_id, dir) for (p_id, (_, p_path)) in dir_pipelines.items()})
    assert dir.exists()

In [ ]:
# doesn't do anything unless include_composite_model = True
composite_model_variant = "currently_active_model"

patch_yearbook = True
dataset_id = "yearbook_test"
eval_handler = "periodic-delta+-1y"
metric = "Accuracy"
include_composite_model = True

In [ ]:
pipelines = {p_id: (pname, p_path) for p_id, (pname, p_path) in pipelines.items()}
pipeline_ids = pipelines.keys()
pipeline_ids = [
    y
    for y, _ in [
        # (20, 'yearbook_timetrigger_40y'),
        (23, "yearbook_timetrigger_25y"),
        (24, "yearbook_timetrigger_15y"),
        (25, "yearbook_timetrigger_10y"),
        (26, "yearbook_timetrigger_5y"),
        (27, "yearbook_timetrigger_4y"),
        (29, "yearbook_timetrigger_3y"),
        (31, "yearbook_timetrigger_2y"),
        (33, "yearbook_timetrigger_1y"),
        (21, "yearbook_dataamount_250"),
        (30, "yearbook_dataamount_500"),
        (32, "yearbook_dataamount_1000"),
        (35, "yearbook_dataamount_2500"),
        (36, "yearbook_dataamount_5000"),
        (37, "yearbook_dataamount_10000"),
        # (38, 'yearbook_dataamount_15000'),
        # (39, 'yearbook_dataamount_30000'),
        # duplicates
        # (666, 'yearbook_dataamount_1000'),
        # (667, 'yearbook_dataamount_250'),
        # (668, 'yearbook_dataamount_2500'),
        # (669, 'yearbook_dataamount_5000'),
        # (670, 'yearbook_dataamount_10000'),
        # (671, 'yearbook_dataamount_500'),
        # (672, 'yearbook_dataamount_15000'),
        # (673, 'yearbook_dataamount_30000'),
        # (63, 'yearbook_drifttrigger_mmd-0.09_int100_win1d'),
        # (64, 'yearbook_drifttrigger_mmd-0.07_int100_win1d'),
        # (65, 'yearbook_drifttrigger_mmd-0.12_int100_win1d'),
        # (66, 'yearbook_drifttrigger_mmd-0.15_int100_win1d'),
        # (67, 'yearbook_drifttrigger_mmd-0.03_int100_win1d'),
        # (68, 'yearbook_drifttrigger_mmd-0.05_int100_win1d'),
        # (69, 'yearbook_drifttrigger_mmd-0.12_int100_win4d'),
        # (70, 'yearbook_drifttrigger_mmd-0.2_int100_win1d'),
        # (71, 'yearbook_drifttrigger_mmd-0.4_int100_win1d'),
        # (72, 'yearbook_drifttrigger_mmd-0.15_int100_win4d'),
        # (73, 'yearbook_drifttrigger_mmd-0.09_int100_win4d'),
        # (74, 'yearbook_drifttrigger_mmd-0.07_int100_win4d'),
        # (75, 'yearbook_drifttrigger_mmd-0.12_int100_win10d'),
        # (76, 'yearbook_drifttrigger_mmd-0.03_int100_win4d'),
        # (77, 'yearbook_drifttrigger_mmd-0.05_int100_win4d'),
        # (78, 'yearbook_drifttrigger_mmd-0.15_int100_win10d'),
        # (79, 'yearbook_drifttrigger_mmd-0.4_int100_win4d'),
        # (80, 'yearbook_drifttrigger_mmd-0.2_int100_win4d'),
        (81, "yearbook_drifttrigger_mmd-0.12_int250_win1d"),
        # (82, 'yearbook_drifttrigger_mmd-0.09_int100_win10d'),
        # (83, 'yearbook_drifttrigger_mmd-0.07_int100_win10d'),
        (84, "yearbook_drifttrigger_mmd-0.15_int250_win1d"),
        (85, "yearbook_drifttrigger_mmd-0.12_int250_win4d"),
        # (86, 'yearbook_drifttrigger_mmd-0.03_int100_win10d'),
        # (87, 'yearbook_drifttrigger_mmd-0.05_int100_win10d'),
        (88, "yearbook_drifttrigger_mmd-0.15_int250_win4d"),
        (89, "yearbook_drifttrigger_mmd-0.12_int250_win10d"),
        # (90, 'yearbook_drifttrigger_mmd-0.2_int100_win10d'),
        (91, "yearbook_drifttrigger_mmd-0.15_int250_win10d"),
        # (92, 'yearbook_drifttrigger_mmd-0.4_int100_win10d'),
        # (93, 'yearbook_drifttrigger_mmd-0.12_int500_win1d'),
        # (94, 'yearbook_drifttrigger_mmd-0.15_int500_win1d'),
        (95, "yearbook_drifttrigger_mmd-0.07_int250_win1d"),
        # (96, 'yearbook_drifttrigger_mmd-0.12_int500_win4d'),
        (97, "yearbook_drifttrigger_mmd-0.09_int250_win1d"),
        # (98, 'yearbook_drifttrigger_mmd-0.15_int500_win4d'),
        (99, "yearbook_drifttrigger_mmd-0.05_int250_win1d"),
        (100, "yearbook_drifttrigger_mmd-0.03_int250_win1d"),
        # (101, 'yearbook_drifttrigger_mmd-0.12_int500_win10d'),
        # (102, 'yearbook_drifttrigger_mmd-0.15_int500_win10d'),
        (103, "yearbook_drifttrigger_mmd-0.2_int250_win1d"),
        (104, "yearbook_drifttrigger_mmd-0.4_int250_win1d"),
        # (105, 'yearbook_drifttrigger_mmd-0.12_int1000_win1d'),
        # (106, 'yearbook_drifttrigger_mmd-0.15_int1000_win1d'),
        (107, "yearbook_drifttrigger_mmd-0.07_int250_win4d"),
        # (108, 'yearbook_drifttrigger_mmd-0.12_int1000_win4d'),
        (109, "yearbook_drifttrigger_mmd-0.09_int250_win4d"),
        # (110, 'yearbook_drifttrigger_mmd-0.15_int1000_win4d'),
        # (111, 'yearbook_drifttrigger_mmd-0.12_int1000_win10d'),
        (112, "yearbook_drifttrigger_mmd-0.05_int250_win4d"),
        (113, "yearbook_drifttrigger_mmd-0.03_int250_win4d"),
        # (114, 'yearbook_drifttrigger_mmd-0.15_int1000_win10d'),
        (115, "yearbook_drifttrigger_mmd-0.2_int250_win4d"),
        (116, "yearbook_drifttrigger_mmd-0.4_int250_win4d"),
        (117, "yearbook_drifttrigger_mmd-0.07_int250_win10d"),
        (118, "yearbook_drifttrigger_mmd-0.09_int250_win10d"),
        (119, "yearbook_drifttrigger_mmd-0.05_int250_win10d"),
        # (122, 'yearbook_drifttrigger_mmd-0.09_int500_win1d'),
        (123, "yearbook_drifttrigger_mmd-0.2_int250_win10d"),
        # (126, 'yearbook_drifttrigger_mmd-0.09_int500_win4d'),
        # (127, 'yearbook_drifttrigger_mmd-0.07_int500_win1d'),
        # (132, 'yearbook_drifttrigger_mmd-0.05_int500_win1d'),
        # (133, 'yearbook_drifttrigger_mmd-0.4_int500_win10d'),
        # (136, 'yearbook_drifttrigger_mmd-0.4_int1000_win1d'),
        # (137, 'yearbook_drifttrigger_mmd-0.2_int500_win1d'),
        # (138, 'yearbook_drifttrigger_mmd-0.09_int1000_win4d'),
        # (139, 'yearbook_drifttrigger_mmd-0.07_int500_win4d'),
        # (144, 'yearbook_drifttrigger_mmd-0.4_int1000_win10d'),
        # (145, 'yearbook_drifttrigger_mmd-0.05_int500_win4d'),
        # (146, 'yearbook_drifttrigger_mmd-0.2_int500_win4d'),
        # (147, 'yearbook_drifttrigger_mmd-0.07_int500_win10d'),
        # (148, 'yearbook_drifttrigger_mmd-0.05_int500_win10d'),
        # (149, 'yearbook_drifttrigger_mmd-0.2_int500_win10d'),
        # (150, 'yearbook_drifttrigger_mmd-0.07_int1000_win1d'),
        # (151, 'yearbook_drifttrigger_mmd-0.05_int1000_win1d'),
        # (152, 'yearbook_drifttrigger_mmd-0.2_int1000_win1d'),
        # (153, 'yearbook_drifttrigger_mmd-0.07_int1000_win4d'),
        # (154, 'yearbook_drifttrigger_mmd-0.05_int1000_win4d'),
        # (155, 'yearbook_drifttrigger_mmd-0.2_int1000_win4d'),
        # (156, 'yearbook_drifttrigger_mmd-0.07_int1000_win10d'),
        # (157, 'yearbook_drifttrigger_mmd-0.05_int1000_win10d'),
        # (158, 'yearbook_drifttrigger_mmd-0.2_int1000_win10d'),
        (159, "yearbook_drifttrigger_mmd-0.03_int250_win10d"),
        # (160, 'yearbook_drifttrigger_mmd-0.03_int1000_win1d'),
        # (161, 'yearbook_drifttrigger_mmd-0.4_int500_win4d'),
        # (162, 'yearbook_drifttrigger_mmd-0.03_int500_win10d'),
        # (163, 'yearbook_drifttrigger_mmd-0.4_int1000_win4d'),
        # (164, 'yearbook_drifttrigger_mmd-0.09_int1000_win10d'),
        # (165, 'yearbook_drifttrigger_mmd-0.03_int500_win1d'),
        (166, "yearbook_drifttrigger_mmd-0.4_int250_win10d"),
        # (167, 'yearbook_drifttrigger_mmd-0.09_int1000_win1d'),
        # (168, 'yearbook_drifttrigger_mmd-0.09_int500_win10d'),
        # (169, 'yearbook_drifttrigger_mmd-0.03_int500_win4d'),
        # (170, 'yearbook_drifttrigger_mmd-0.4_int500_win1d'),
        # (171, 'yearbook_drifttrigger_mmd-0.03_int1000_win10d'),
        # (172, 'yearbook_drifttrigger_mmd-0.03_int1000_win4d'),
        # (329, 'yearbook_drifttrigger_mmd-quant-0.05-10_int500_win4d'),
        # (330, 'yearbook_drifttrigger_mmd-quant-0.05-20_int500_win4d\n'),
        # (331, 'yearbook_drifttrigger_mmd-quant-0.05-30_int100_win4d'),
        # (332, 'yearbook_drifttrigger_mmd-quant-0.1-10_int500_win4d'),
        # (333, 'yearbook_drifttrigger_mmd-quant-0.15-20_int500_win4d'),
        # (334, 'yearbook_drifttrigger_mmd-quant-0.15-10_int500_win4d'),
        # (335, 'yearbook_drifttrigger_mmd-quant-0.1-20_int500_win4d'),
        # (336, 'yearbook_drifttrigger_mmd-quant-0.3-20_int500_win4d'),
        # (337, 'yearbook_drifttrigger_mmd-quant-0.1-30_int500_win4d'),
        # (338, 'yearbook_drifttrigger_mmd-quant-0.3-10_int500_win4d'),
        # (339, 'yearbook_drifttrigger_mmd-rollavg-0.05-20_int500_win4d'),
        (340, "yearbook_drifttrigger_mmd-quant-0.1-10_int250_win4d"),
        # (341, 'yearbook_drifttrigger_mmd-quant-0.15-30_int100_win4d'),
        # (342, 'yearbook_drifttrigger_mmd-rollavg-0.05-10_int500_win4d'),
        # (343, 'yearbook_drifttrigger_mmd-rollavg-0.2-20_int500_win4d'),
        # (344, 'yearbook_drifttrigger_mmd-rollavg-0.2-10_int500_win4d'),
        (345, "yearbook_drifttrigger_mmd-quant-0.1-20_int250_win4d"),
        # (346, 'yearbook_drifttrigger_mmd-rollavg-0.5-20_int500_win4d'),
        # (347, 'yearbook_drifttrigger_mmd-rollavg-0.5-10_int500_win4d'),
        # (348, 'yearbook_drifttrigger_mmd-rollavg-1.0-20_int500_win4d'),
        (349, "yearbook_drifttrigger_mmd-quant-0.1-30_int250_win4d"),
        # (350, 'yearbook_drifttrigger_mmd-rollavg-1.0-10_int500_win4d'),
        # (351, 'yearbook_drifttrigger_mmd-rollavg-2.0-20_int500_win4d'),
        # (352, 'yearbook_drifttrigger_mmd-quant-0.3-30_int100_win4d'),
        (353, "yearbook_drifttrigger_mmd-quant-0.05-20_int250_win4d"),
        # (354, 'yearbook_drifttrigger_mmd-rollavg-2.0-10_int500_win4d'),
        # (355, 'yearbook_drifttrigger_mmd-quant-0.1-10_int100_win4d'),
        (356, "yearbook_drifttrigger_mmd-quant-0.05-10_int250_win4d"),
        (357, "yearbook_drifttrigger_mmd-quant-0.15-20_int250_win4d"),
        (358, "yearbook_drifttrigger_mmd-quant-0.15-10_int250_win4d"),
        (359, "yearbook_drifttrigger_mmd-quant-0.3-20_int250_win4d"),
        # (360, 'yearbook_drifttrigger_mmd-rollavg-0.05-30_int100_win4d'),
        # (361, 'yearbook_drifttrigger_mmd-quant-0.1-20_int100_win4d'),
        (362, "yearbook_drifttrigger_mmd-quant-0.3-10_int250_win4d"),
        (363, "yearbook_drifttrigger_mmd-rollavg-0.05-20_int250_win4d"),
        (364, "yearbook_drifttrigger_mmd-rollavg-0.05-10_int250_win4d"),
        (365, "yearbook_drifttrigger_mmd-rollavg-0.2-20_int250_win4d"),
        # (366, 'yearbook_drifttrigger_mmd-quant-0.1-30_int100_win4d'),
        # (367, 'yearbook_drifttrigger_mmd-rollavg-0.2-30_int100_win4d'),
        (368, "yearbook_drifttrigger_mmd-rollavg-0.2-10_int250_win4d"),
        (369, "yearbook_drifttrigger_mmd-rollavg-0.5-20_int250_win4d"),
        (370, "yearbook_drifttrigger_mmd-rollavg-1.0-20_int250_win4d"),
        (371, "yearbook_drifttrigger_mmd-rollavg-0.5-10_int250_win4d"),
        (372, "yearbook_drifttrigger_mmd-rollavg-2.0-20_int250_win4d"),
        (373, "yearbook_drifttrigger_mmd-rollavg-1.0-10_int250_win4d"),
        # (374, 'yearbook_drifttrigger_mmd-rollavg-0.5-30_int100_win4d'),
        # (375, 'yearbook_drifttrigger_mmd-quant-0.05-20_int100_win4d'),
        (376, "yearbook_drifttrigger_mmd-rollavg-2.0-10_int250_win4d"),
        # (377, 'yearbook_drifttrigger_mmd-quant-0.05-10_int100_win4d'),
        # (378, 'yearbook_drifttrigger_mmd-rollavg-1.0-30_int100_win4d'),
        # (379, 'yearbook_drifttrigger_mmd-quant-0.15-20_int100_win4d'),
        # (380, 'yearbook_drifttrigger_mmd-quant-0.15-10_int100_win4d'),
        # (381, 'yearbook_drifttrigger_mmd-rollavg-2.0-30_int100_win4d'),
        # (382, 'yearbook_drifttrigger_mmd-quant-0.3-20_int100_win4d'),
        (383, "yearbook_drifttrigger_mmd-quant-0.05-30_int250_win4d"),
        # (384, 'yearbook_drifttrigger_mmd-quant-0.3-10_int100_win4d'),
        (385, "yearbook_drifttrigger_mmd-quant-0.15-30_int250_win4d"),
        (386, "yearbook_drifttrigger_mmd-quant-0.3-30_int250_win4d"),
        # (387, 'yearbook_drifttrigger_mmd-rollavg-0.05-20_int100_win4d'),
        (388, "yearbook_drifttrigger_mmd-rollavg-0.05-30_int250_win4d"),
        # (389, 'yearbook_drifttrigger_mmd-rollavg-0.05-10_int100_win4d'),
        (390, "yearbook_drifttrigger_mmd-rollavg-0.2-30_int250_win4d"),
        # (391, 'yearbook_drifttrigger_mmd-rollavg-0.2-20_int100_win4d'),
        (392, "yearbook_drifttrigger_mmd-rollavg-0.5-30_int250_win4d"),
        (393, "yearbook_drifttrigger_mmd-rollavg-1.0-30_int250_win4d"),
        # (394, 'yearbook_drifttrigger_mmd-rollavg-0.2-10_int100_win4d'),
        (395, "yearbook_drifttrigger_mmd-rollavg-2.0-30_int250_win4d"),
        # (396, 'yearbook_drifttrigger_mmd-rollavg-0.5-20_int100_win4d'),
        # (397, 'yearbook_drifttrigger_mmd-quant-0.05-30_int500_win4d'),
        # (398, 'yearbook_drifttrigger_mmd-quant-0.15-30_int500_win4d'),
        # (399, 'yearbook_drifttrigger_mmd-rollavg-0.5-10_int100_win4d'),
        # (400, 'yearbook_drifttrigger_mmd-quant-0.3-30_int500_win4d'),
        # (401, 'yearbook_drifttrigger_mmd-rollavg-1.0-20_int100_win4d'),
        # (402, 'yearbook_drifttrigger_mmd-rollavg-0.05-30_int500_win4d'),
        # (403, 'yearbook_drifttrigger_mmd-rollavg-0.2-30_int500_win4d'),
        # (404, 'yearbook_drifttrigger_mmd-rollavg-1.0-10_int100_win4d'),
        # (405, 'yearbook_drifttrigger_mmd-rollavg-0.5-30_int500_win4d'),
        # (406, 'yearbook_drifttrigger_mmd-rollavg-2.0-20_int100_win4d'),
        # (407, 'yearbook_drifttrigger_mmd-rollavg-1.0-30_int500_win4d'),
        # (408, 'yearbook_drifttrigger_mmd-rollavg-2.0-30_int500_win4d'),
        # (409, 'yearbook_drifttrigger_mmd-rollavg-2.0-10_int100_win4d'),
        # (683,
        # 'yearbook_performancetrigger_num_misclass-1500-exp-0.85-red-True--int250y'),
        # (685,
        # 'yearbook_performancetrigger_num_misclass-1500-exp-0.85-red-False--int250y'),
        (686, "yearbook_performancetrigger_num_misclass-1500-exp-0.9-red-True--int250y"),
        (687, "yearbook_performancetrigger_num_misclass-1500-exp-0.9-red-False--int250y"),
        # (688,
        # 'yearbook_performancetrigger_num_misclass-1500-exp-0.95-red-True--int250y'),
        # (704,
        # 'yearbook_performancetrigger_num_misclass-200-exp-0.85-red-False--int250y'),
        # (727,
        # 'yearbook_performancetrigger_num_misclass-1500-exp-0.95-red-False--int250y'),
        # (728,
        # 'yearbook_performancetrigger_num_misclass-1000-exp-0.85-red-True--int250y'),
        # (729,
        # 'yearbook_performancetrigger_num_misclass-1000-exp-0.85-red-False--int250y'),
        # (730,
        # 'yearbook_performancetrigger_num_misclass-50-exp-0.85-red-True--int250y'),
        (731, "yearbook_performancetrigger_num_misclass-1000-exp-0.9-red-True--int250y"),
        # (732,
        # 'yearbook_performancetrigger_num_misclass-50-exp-0.85-red-False--int250y'),
        (733, "yearbook_performancetrigger_num_misclass-1000-exp-0.9-red-False--int250y"),
        (734, "yearbook_performancetrigger_num_misclass-50-exp-0.9-red-True--int250y"),
        # (735,
        # 'yearbook_performancetrigger_num_misclass-1000-exp-0.95-red-True--int250y'),
        (736, "yearbook_performancetrigger_num_misclass-50-exp-0.9-red-False--int250y"),
        # (737,
        # 'yearbook_performancetrigger_num_misclass-1000-exp-0.95-red-False--int250y'),
        # (738,
        # 'yearbook_performancetrigger_num_misclass-50-exp-0.95-red-True--int250y'),
        # (739,
        # 'yearbook_performancetrigger_num_misclass-500-exp-0.85-red-True--int250y'),
        # (740,
        # 'yearbook_performancetrigger_num_misclass-50-exp-0.95-red-False--int250y'),
        # (741,
        # 'yearbook_performancetrigger_num_misclass-500-exp-0.85-red-False--int250y'),
        (742, "yearbook_performancetrigger_num_misclass-500-exp-0.9-red-True--int250y"),
        (743, "yearbook_performancetrigger_num_misclass-500-exp-0.9-red-False--int250y"),
        # (744,
        # 'yearbook_performancetrigger_num_misclass-500-exp-0.95-red-True--int250y'),
        # (746,
        # 'yearbook_performancetrigger_num_misclass-500-exp-0.95-red-False--int250y'),
        # (747,
        # 'yearbook_performancetrigger_num_misclass-200-exp-0.85-red-True--int250y'),
        (749, "yearbook_performancetrigger_num_misclass-200-exp-0.9-red-True--int250y"),
        (751, "yearbook_performancetrigger_num_misclass-200-exp-0.9-red-False--int250y"),
        # (753,
        # 'yearbook_performancetrigger_num_misclass-200-exp-0.95-red-True--int250y'),
        # (754,
        # 'yearbook_performancetrigger_num_misclass-200-exp-0.95-red-False--int250y'),
        # (755,
        # 'yearbook_performancetrigger_num_misclass-100-exp-0.85-red-True--int250y'),
        # (757,
        # 'yearbook_performancetrigger_num_misclass-100-exp-0.85-red-False--int250y'),
        (758, "yearbook_performancetrigger_num_misclass-100-exp-0.9-red-True--int250y"),
        (759, "yearbook_performancetrigger_num_misclass-100-exp-0.9-red-False--int250y"),
        # (760,
        # 'yearbook_performancetrigger_num_misclass-100-exp-0.95-red-True--int250y'),
        # (761,
        # 'yearbook_performancetrigger_num_misclass-100-exp-0.95-red-False--int250y'),
        # (410, 'yearbook_performancetrigger_static-0.7-int100y'),
        (411, "yearbook_performancetrigger_static-0.7-int250y"),
        # (412, 'yearbook_performancetrigger_static-0.7-int500y'),
        # (413, 'yearbook_performancetrigger_static-0.75-int500y'),
        (414, "yearbook_performancetrigger_static-0.75-int250y"),
        # (416, 'yearbook_performancetrigger_static-0.8-int500y'),
        # (417, 'yearbook_performancetrigger_static-0.75-int100y'),
        (418, "yearbook_performancetrigger_static-0.8-int250y"),
        # (419, 'yearbook_performancetrigger_static-0.85-int500y'),
        (421, "yearbook_performancetrigger_static-0.85-int250y"),
        # (422, 'yearbook_performancetrigger_static-0.875-int500y'),
        # (423, 'yearbook_performancetrigger_static-0.8-int100y'),
        # (424, 'yearbook_performancetrigger_static-0.9-int500y'),
        (425, "yearbook_performancetrigger_static-0.875-int250y"),
        # (427, 'yearbook_performancetrigger_static-0.925-int500y'),
        # (428, 'yearbook_performancetrigger_static-0.85-int100y'),
        (429, "yearbook_performancetrigger_static-0.9-int250y"),
        # (430, 'yearbook_performancetrigger_static-0.95-int500y'),
        (432, "yearbook_performancetrigger_static-0.925-int250y"),
        # (433, 'yearbook_performancetrigger_dynamic-quant-0.05-10-int500y'),
        # (434, 'yearbook_performancetrigger_static-0.875-int100y'),
        # (436, 'yearbook_performancetrigger_dynamic-quant-0.05-20-int500y'),
        (437, "yearbook_performancetrigger_static-0.95-int250y"),
        # (438, 'yearbook_performancetrigger_dynamic-quant-0.05-30-int500y'),
        # (440, 'yearbook_performancetrigger_dynamic-quant-0.15-10-int500y'),
        # (441, 'yearbook_performancetrigger_static-0.9-int100y'),
        # (442, 'yearbook_performancetrigger_dynamic-quant-0.05-10-int250y'),
        # (443, 'yearbook_performancetrigger_dynamic-quant-0.15-20-int500y'),
        (445, "yearbook_performancetrigger_dynamic-quant-0.05-20-int250y"),
        # (446, 'yearbook_performancetrigger_dynamic-quant-0.15-30-int500y'),
        # (447, 'yearbook_performancetrigger_dynamic-quant-0.3-10-int500y'),
        # (448, 'yearbook_performancetrigger_dynamic-quant-0.05-30-int250y'),
        # (450, 'yearbook_performancetrigger_static-0.925-int100y'),
        # (451, 'yearbook_performancetrigger_dynamic-quant-0.3-20-int500y'),
        # (452, 'yearbook_performancetrigger_dynamic-quant-0.15-10-int250y'),
        # (454, 'yearbook_performancetrigger_dynamic-quant-0.3-30-int500y'),
        (455, "yearbook_performancetrigger_dynamic-quant-0.15-20-int250y"),
        # (458, 'yearbook_performancetrigger_dynamic-quant-0.15-30-int250y'),
        # (459, 'yearbook_performancetrigger_static-0.95-int100y'),
        # (463, 'yearbook_performancetrigger_dynamic-quant-0.3-10-int250y'),
        # (464, 'yearbook_performancetrigger_dynamic-rollavg-0.05-10-int500y'),
        # (465, 'yearbook_performancetrigger_dynamic-rollavg-0.05-20-int500y'),
        (467, "yearbook_performancetrigger_dynamic-quant-0.3-20-int250y"),
        # (468, 'yearbook_performancetrigger_dynamic-rollavg-0.05-30-int500y'),
        # (469, 'yearbook_performancetrigger_dynamic-rollavg-0.1-10-int500y'),
        # (471, 'yearbook_performancetrigger_dynamic-quant-0.3-30-int250y'),
        # (472, 'yearbook_performancetrigger_dynamic-rollavg-0.1-20-int500y'),
        # (473, 'yearbook_performancetrigger_dynamic-quant-0.05-10-int100y'),
        # (474, 'yearbook_performancetrigger_dynamic-rollavg-0.1-30-int500y'),
        # (475, 'yearbook_performancetrigger_dynamic-rollavg-0.2-10-int500y'),
        # (478, 'yearbook_performancetrigger_dynamic-rollavg-0.2-20-int500y'),
        # (479, 'yearbook_performancetrigger_dynamic-rollavg-0.2-30-int500y'),
        (481, "yearbook_performancetrigger_dynamic-quant-0.05-20-int100y"),
        # (483, 'yearbook_performancetrigger_dynamic-rollavg-0.3-10-int500y'),
        # (484, 'yearbook_performancetrigger_dynamic-rollavg-0.3-20-int500y'),
        # (486, 'yearbook_performancetrigger_dynamic-rollavg-0.3-30-int500y'),
        # (489, 'yearbook_performancetrigger_dynamic-quant-0.05-30-int100y'),
        # (491, 'yearbook_performancetrigger_dynamic-rollavg-0.05-10-int250y'),
        (494, "yearbook_performancetrigger_dynamic-rollavg-0.05-20-int250y"),
        # (497, 'yearbook_performancetrigger_dynamic-quant-0.15-10-int100y'),
        # (499, 'yearbook_performancetrigger_dynamic-rollavg-0.05-30-int250y'),
        # (503, 'yearbook_performancetrigger_dynamic-rollavg-0.1-10-int250y'),
        (506, "yearbook_performancetrigger_dynamic-rollavg-0.1-20-int250y"),
        (507, "yearbook_performancetrigger_dynamic-quant-0.15-20-int100y"),
        # (509, 'yearbook_performancetrigger_dynamic-rollavg-0.1-30-int250y'),
        # (513, 'yearbook_performancetrigger_dynamic-rollavg-0.2-10-int250y'),
        (516, "yearbook_performancetrigger_dynamic-rollavg-0.2-20-int250y"),
        # (519, 'yearbook_performancetrigger_dynamic-quant-0.15-30-int100y'),
        # (521, 'yearbook_performancetrigger_dynamic-rollavg-0.2-30-int250y'),
        # (524, 'yearbook_performancetrigger_dynamic-rollavg-0.3-10-int250y'),
        # (527, 'yearbook_performancetrigger_dynamic-rollavg-0.3-20-int250y'),
        # (529, 'yearbook_performancetrigger_dynamic-rollavg-0.3-30-int250y'),
        # (530, 'yearbook_performancetrigger_dynamic-quant-0.3-10-int100y'),
        (539, "yearbook_performancetrigger_dynamic-quant-0.3-20-int100y"),
        # (548, 'yearbook_performancetrigger_dynamic-quant-0.3-30-int100y'),
        # (818, 'yearbook_costtrigger_data_inc_int250_exch15552000'),
        # (819, 'yearbook_costtrigger_data_inc_int250_exch13824000'),
        (821, "yearbook_costtrigger_avoidable_misclass_int250_exch86400000_redFalse"),
        # (822, 'yearbook_costtrigger_data_inc_int250_exch12096000'),
        # (825, 'yearbook_costtrigger_data_inc_int250_exch10368000'),
        # (835, 'yearbook_costtrigger_data_inc_int250_exch129600000'),
        # (836, 'yearbook_costtrigger_data_inc_int250_exch34560000'),
        # (837, 'yearbook_costtrigger_data_inc_int250_exch25920000'),
        # (838, 'yearbook_costtrigger_data_inc_int250_exch4320000'),
        #  (821, 'yearbook_costtrigger_avoidable_misclass_int250_exch86400000_redFalse'),
        (839, "yearbook_costtrigger_avoidable_misclass_int250_exch86400000_redFalse"),
        (840, "yearbook_costtrigger_avoidable_misclass_int250_exch864000_redFalse"),
        (841, "yearbook_costtrigger_avoidable_misclass_int250_exch864000_redFalse"),
        (842, "yearbook_costtrigger_avoidable_misclass_int250_exch864.0_redFalse"),
        #  (843, 'yearbook_costtrigger_avoidable_misclass_int250_exch8640000_redFalse'),
        (844, "yearbook_costtrigger_avoidable_misclass_int250_exch8640.0_redFalse"),
        #  (846, 'yearbook_costtrigger_avoidable_misclass_int250_exch8640000000_redFalse'),
        #  (847, 'yearbook_costtrigger_avoidable_misclass_int250_exch864000000_redFalse'),
        (848, "yearbook_costtrigger_avoidable_misclass_int250_exch64800.0_redFalse"),
        #  (849, 'yearbook_costtrigger_avoidable_misclass_int250_exch43200.0_redFalse'),
        (850, "yearbook_costtrigger_avoidable_misclass_int250_exch21600.0_redFalse"),
        #  (851, 'yearbook_costtrigger_avoidable_misclass_int250_exch4320000000_redFalse')]
    ]
]

[(p_id, pname) for p_id, (pname, _) in pipelines.items() if p_id in pipeline_ids]

# Wrangle data

In [ ]:
list_df_eval_single: list[pd.DataFrame] = []
df_leaf_list: list[pd.DataFrame] = []

for pipeline_id in pipeline_ids:
    logs = pipeline_logs[pipeline_id]
    df_leaf_single = pipeline_leaf_times_df(logs, use_traintime_patch_at_trainer=True, pipeline_id=pipeline_id)
    df_leaf_single["pipeline_id"] = pipeline_id
    df_leaf_list.append(df_leaf_single)

    _, _, df_eval_single = dfs_models_and_evals(
        pipeline_logs[pipeline_id], df_leaf_single["sample_time"].max(), pipelines[pipeline_id][0]
    )
    df_eval_single["pipeline_id"] = pipeline_id
    list_df_eval_single.append(df_eval_single)

df_adjusted = pd.concat(list_df_eval_single)
df_adjusted

df_leaf = pd.concat(df_leaf_list)
df_leaf

In [ ]:
print(df_leaf["id"].unique())
assert set(df_leaf["id"].unique()) == {
    "TRAIN",
    "INIT_CLUSTER_CONNECTION",
    "EVALUATE_TRIGGER_POLICY",
    "INFORM_SELECTOR_REMAINING_DATA",
    "INFORM_SELECTOR_ABOUT_TRIGGER",
    "TRAINING_COMPLETED",
    "STORE_TRAINED_MODEL",
    "EVALUATE",
    "DONE",
}

In [ ]:
df_adjusted = df_adjusted[
    (df_adjusted["dataset_id"] == dataset_id)
    & (df_adjusted["eval_handler"] == eval_handler)
    & (df_adjusted["metric"] == metric)
]

# in percent (0-100)
df_adjusted["value"] = df_adjusted["value"] * 100
df_adjusted

In [ ]:
if patch_yearbook:
    for column in ["interval_start", "interval_center", "interval_end"]:
        patch_yearbook_time(df_adjusted, column)
    patch_yearbook_time(df_leaf, "sample_time")

In [ ]:
df_adjusted = df_adjusted.sort_values(by=["interval_center"])

In [ ]:
# Reduce to composite models
df_adjusted = df_adjusted[df_adjusted[composite_model_variant]]
df_adjusted[composite_model_variant].unique()

In [ ]:
# reduce evaluation interval to interval where all policies have evaluations
min_active_eval_center_per_pipeline = (
    df_adjusted[df_adjusted[composite_model_variant]].groupby("pipeline_ref")["interval_center"].min()
)
maximum_min = min_active_eval_center_per_pipeline.max()
print(maximum_min, min_active_eval_center_per_pipeline)

assert maximum_min < pd.Timestamp("1962-01-01")

df_adjusted = df_adjusted[df_adjusted["interval_center"] >= maximum_min]
df_adjusted["interval_center"].unique()

In [ ]:
df_adjusted["interval_center"] = df_adjusted["interval_center"].astype(str).str.split("-").str[0]

In [ ]:
df_adjusted

In [ ]:
# Aggregate metrics to a scalar value per pipeline
mean_accuracies = df_aggregate_eval_metric(
    df_adjusted,
    group_by=["pipeline_id", "pipeline_ref", "metric"],
    in_col="value",
    out_col="metric_value",
    aggregate_func="mean",
)
mean_accuracies

In [ ]:
df_triggers = df_leaf[df_leaf["id"] == PipelineStage.TRAIN.name]

In [ ]:
df_triggers = df_leaf[df_leaf["id"] == PipelineStage.TRAIN.name]
df_triggers = df_triggers[df_triggers["sample_time"] > maximum_min]
df_triggers

In [ ]:
# Find number of trigger per pipeline that are after maximum_min

# before the cutoff there was one trigger (equivalent to start of our reduced dataset): +1
num_triggers = df_triggers.groupby("pipeline_id").aggregate(count=("id", "count"), sum_duration=("duration", "sum"))
num_triggers["count"] += 1
num_triggers

In [ ]:
num_triggers

In [ ]:
merged = num_triggers.merge(mean_accuracies, on="pipeline_id", how="inner")
assert num_triggers.shape[0] == merged.shape[0]
merged

In [ ]:
def create_type(x: str):
    if "year" in x:
        return "time"
    elif "samples" in x:
        return "amount"
    elif "d" in x:
        return "drift"
    else:
        return "unknown"


merged["type"] = merged["pipeline_ref"].apply(lambda x: create_type(x))
merged

In [ ]:
renamed = merged.copy()

# renamed = merged[
#     merged["pipeline_id"].isin(
#         [
#             # # static thresholds
#             # 113,  # 0.03
#             # 112,  # 0.05
#             # 107,  # 0.07
#             # 109,  # 0.09
#             # 85,  # 0.12
#             # # dyn quantile
#             # 353,  # % 0.05
#             # 345,  # % 0.10
#             # 357,  # % 0.15
#             # # dyn roll. avg
#             # 372,  # Δ 2.0
#             # 370,  # Δ 1.0
#             # 369,  # Δ 0.5
#             # 363,  # Δ 0.05
#         ]
#     )
# ].copy()
renamed["Trigger SubType"] = renamed["pipeline_ref"].apply(
    lambda x: (
        "DataAmount"
        if "dataamount" in x
        else (
            "Time"
            if "time" in x
            else (
                (
                    "Static"
                    if "_mmd-0" in x
                    else ("Quantile" if "quant" in x else ("Rolling Avg." if "roll" in x else ("unknown")))
                )
                if "drift" in x
                else (
                    (
                        "Static"
                        if "static" in x
                        else (
                            "Quantile"
                            if "quant" in x
                            else (
                                "Rolling Avg."
                                if "roll" in x
                                else ("AvoidableMisclass" if "num_misclass" in x else ("unknown"))
                            )
                        )
                    )
                    if "performancetrigger" in x
                    else (
                        "DataIncorporationLatency"
                        if "data_inc" in x
                        else ("AvoidableMisclass" if "avoidable" in x else ("unknown"))
                    )
                )
            )
        )
    )
)
renamed["Trigger Type"] = renamed["pipeline_ref"].apply(
    lambda x: (
        "Simple"
        if "dataamount" in x
        else (
            "Simple"
            if "time" in x
            else (
                "DataDrift"
                if "drift" in x
                else ("Performance" if "performancetrigger" in x else ("Cost" if "costtrigger" in x else ("unknown")))
            )
        )
    )
)

# assert no unknowns and DataIncorporationLatency
assert not renamed["Trigger Type"].str.contains("unknown").any()
assert not renamed["Trigger SubType"].str.contains("unknown").any()
assert not renamed["Trigger SubType"].str.contains("DataIncorporationLatency").any()

renamed["Trigger Type"] = pd.Categorical(
    renamed["Trigger Type"], categories=["Simple", "DataDrift", "Performance", "Cost"], ordered=True
)

renamed["Trigger SubType"] = pd.Categorical(
    renamed["Trigger SubType"],
    categories=["DataAmount", "Time", "Static", "Quantile", "Rolling Avg.", "AvoidableMisclass"],
    ordered=True,
)

renamed = renamed.sort_values(by=["Trigger Type", "Trigger SubType", "pipeline_id"])

In [ ]:
fig = plot_tradeoff_scatter(
    renamed,
    x="count",
    y="metric_value",
    hue="Trigger Type",
    style="Trigger SubType",
    x_label="Number of Triggers",
    y_label="Mean Accuracy %",
    height_factor=0.8,
    width_factor=0.9,
    manual_legend_title=False,
    legend_ncol=2,
)

save_plot(fig, "_all_tradeoff_yearbook_triggers_performance")

In [ ]:
in_minutes = renamed.copy()
in_minutes["sum_duration"] = in_minutes["sum_duration"] / 60

fig = plot_tradeoff_scatter(
    in_minutes,
    x="sum_duration",
    y="metric_value",
    hue="Trigger Type",
    style="Trigger SubType",
    x_label="Total Cost (Minutes)",
    y_label="Mean Accuracy %",
    height_factor=0.8,
    width_factor=0.9,
    manual_legend_title=False,
    legend_ncol=2,
)

save_plot(fig, "_all_tradeoff_yearbook_cost_performance")

In [ ]:
fig = plot_tradeoff_scatter(
    renamed,
    x="count",
    y="sum_duration",
    hue="Trigger Type",
    style="Trigger SubType",
    x_label="Number of Triggers",
    y_label="Total Cost (seconds)",
    height_factor=1.5,
    width_factor=1.8,
    manual_legend_title=False,
    legend_ncol=2,
)

# save_plot(fig, "tradeoff_drift_yearbook_triggers_cost")